In [1]:
from bs4 import BeautifulSoup
import requests
import time

def scraping(topic):
    url = 'https://news.yahoo.co.jp/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    elements = soup.select('.sc-fKgJPI.fFnFHR')
    
    articles_dict = []
    topic = topic 
    #トピックのURL取得
    for i, element in enumerate(elements):
        if topic in element.text:
            categories_num = i
    
    element = elements[categories_num]
    topic_url= element.a['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    elements = soup.select('.sc-fXazdy.UjHkE')
    elements = elements[:5]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        url = element.a['href']
        title = element.text[1:]
        news_urls.append(url)
        news_titles.append(title)
        time.sleep(1)
    
    news_text = []
    #ニュースの本文を取得
    for news_url in news_urls:
        r = requests.get(news_url)
        soup = BeautifulSoup(r.text, "html.parser")
        elements = soup.select('.sc-hcmsbg.iHLBIg.yjSlinkDirectlink.highLightSearchTarget')

        text = ""

        #ニュース記事の本文が配列で区切られていた場合
        for i in range(0, 8):
            try:
                if (i == 0):
                    text = elements[0].text
                else:
                    text += elements[i].text
            except:
                pass

        #ページが1つだけのとき、エラーが出るため
        try:
            page_element = soup.select('.sc-dYXZXt.jzZUwL')
            page = page_element[0].text         
            page_str = ''.join(filter(str.isdigit, page))
            page_num = int(page_str)
        except:
            page_num = 1
            
        #ページ数が複数のとき、ページ遷移し本文を取得
        if (page_num > 1):
            for i in range(2, page_num + 1):
                news_url_page = news_url + "?page=" + str(page_num)
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                elements = soup.select('.sc-hcmsbg.iHLBIg.yjSlinkDirectlink.highLightSearchTarget')
                #ニュース記事の本文が配列で区切られていた場合
                for i in range(0, 8):
                    try:
                        text += elements[i].text
                    except:
                        pass
                        
        news_text.append(text)
    
        time.sleep(1)
    
    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles
    

In [2]:
def clean_data(articles):
    for i in range(len(articles)):
        articles[i]['title'] = articles[i]['title'].replace('\u3000', '') 
        articles[i]['title'] = articles[i]['title'].lstrip('0123456789')
        articles[i]['text'] = articles[i]['text'].replace('\u3000', '')
        articles[i]['text'] = articles[i]['text'].replace('\n', '')

    return articles

In [3]:
articles = scraping('ライフ')

In [4]:
articles

[]

In [63]:
articles = clean_data(articles)

In [64]:
articles

[{'title': '５メートルのテーブルに座ったプーチンが変わった…反乱後に見せた「奇異な光景」に驚き中央日報日本語版7/3(月)6:50',
  'URL': 'https://news.yahoo.co.jp/articles/d7d5dc9bef45018d2317f816dd6da2718e93d29d',
  'text': '先月２４日に民間軍事会社ワグネルグループの反乱が中断された後、ロシアのプーチン大統領の公開行動が目に見えて増えた。ワグネルグループ創設者のプリゴジン氏がベラルーシのルカシェンコ大統領との交渉により「モスクワ進撃」を止めてからプーチン大統領は４日間全国を飛び回り、大衆に残らず公開された。ＢＢＣは１日、「プーチン氏が１週間全速力で走りあちこちで神出鬼没するように登場した。まるで再選に向けた運動を始めたかのようだ」としてプーチン氏の動きを集中分析した。【写真】落書きしながら余裕ある姿を見せるロシアのプーチン大統領反乱後にプーチン大統領が初めて姿を表わしたのは先月２６日の国民向け演説だった。夜１０時を過ぎた時間にテレビで中継されたこの演説で彼が強調したのは流血事態を防いだという点だった。プーチン大統領は「事件初期から深刻な流血事態を避けようと私の命令により措置が取られた」と強調した。自身の指導力により事態悪化を防いだということだ。また、反乱主導者を非難しながらも「ワグネルグループの指揮官と兵士の大部分は愛国者であることを知っている」と話した。反乱勢力のうち幹部と一般戦闘員を区分したのだ。２７日には公開席上に姿を見せ健在を誇示する本格的な歩みに出た。彼はレッドカーペットが敷かれたクレムリンの野外階段を降りてきて広場で保衛軍と近衛隊の軍人約２５００人を対象に演説をした。ロシア皇帝の戴冠式の行列が歩いた場所だ。ここでプーチン大統領は「みなさんが祖国を激変から救った。事実上内戦を防いだ」として軍を称えた。この日の行事以降プーチン大統領の公開行動が増えた。彼は先月２６日の国民向け演説をはじめ異例の公開席上に登場した。翌２８日には正常業務に復帰したことを見せるようにモスクワを離れ現場に出た。彼が訪問したのはロシア南西部ダゲスタン共和国デルベント。観光発展会議を主宰するための訪問だったが、話題になったのは市民の中にまぎれ込んだプーチン大統領の姿だった